In [28]:
import pandas as pd

df = pd.read_csv('training_data_2018_2020_feb_13_20201.csv').iloc[:,1:66]
df

,Datetime,AIL,hour_of_day,off_peak,on_peak,just_date,day,sin.day,cos.day,sin.hour,...,year,sunlight_avaialbility,AIL_previous_hour,AIL_24h_lagged,AIL_2day_lagged,AIL_3day_lagged,AIL_4day_lagged,AIL_5day_lagged,AIL_6day_lagged,AIL_oneweek_lagged
0,2018-01-01 00:00:00,10221.0,0,1,0,2018-01-01,1,0.719174,0.694830,0.000000,...,2018,0,10571.0,10221.0,10221.0,10221.0,10221.0,10221.0,10221.0,10221.0
1,2018-01-01 01:00:00,10082.0,1,1,0,2018-01-01,1,0.719174,0.694830,0.258819,...,2018,0,10221.0,10082.0,10082.0,10082.0,10082.0,10082.0,10082.0,10082.0
2,2018-01-01 02:00:00,9949.0,2,1,0,2018-01-01,1,0.719174,0.694830,0.500000,...,2018,0,10082.0,9949.0,9949.0,9949.0,9949.0,9949.0,9949.0,9949.0
3,2018-01-01 03:00:00,9886.0,3,1,0,2018-01-01,1,0.719174,0.694830,0.707107,...,2018,0,9949.0,9886.0,9886.0,9886.0,9886.0,9886.0,9886.0,9886.0
4,2018-01-01 04:00:00,9930.0,4,1,0,2018-01-01,1,0.719174,0.694830,0.866025,...,2018,0,9886.0,9930.0,9930.0,9930.0,9930.0,9930.0,9930.0,9930.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27283,2021-02-10 19:00:00,11485.0,19,0,1,2021-02-10,41,0.996832,0.079532,-0.965926,...,2021,0,11665.0,11598.0,11513.0,11271.0,11326.0,11298.0,11165.0,10961.0
27284,2021-02-10 20:00:00,11364.0,20,1,0,2021-02-10,41,0.996832,0.079532,-0.866025,...,2021,0,11485.0,11490.0,11390.0,11127.0,11235.0,11178.0,11057.0,10887.0
27285,2021-02-10 21:00:00,11195.0,21,1,0,2021-02-10,41,0.996832,0.079532,-0.707107,...,2021,0,11364.0,11410.0,11283.0,11093.0,11181.0,11018.0,10860.0,10725.0
27286,2021-02-10 22:00:00,10961.0,22,1,0,2021-02-10,41,0.996832,0.079532,-0.500000,...,2021,0,11195.0,11192.0,11148.0,10958.0,11030.0,10811.0,10590.0,10524.0


In [29]:
df.columns

Index(['Datetime', 'AIL', 'hour_of_day', 'off_peak', 'on_peak', 'just_date',
       'day', 'sin.day', 'cos.day', 'sin.hour', 'cos.hour', 'weekend',
       'monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday',
       'sunday', 'month_0', 'month_1', 'month_2', 'month_3', 'month_4',
       'month_5', 'month_6', 'month_7', 'month_8', 'month_9', 'month_10',
       'month_11', 'hour_0', 'hour_1', 'hour_2', 'hour_3', 'hour_4', 'hour_5',
       'hour_6', 'hour_7', 'hour_8', 'hour_9', 'hour_10', 'hour_11', 'hour_12',
       'hour_13', 'hour_14', 'hour_15', 'hour_16', 'hour_17', 'hour_18',
       'hour_19', 'hour_20', 'hour_21', 'hour_22', 'hour_23', 'year',
       'sunlight_avaialbility', 'AIL_previous_hour', 'AIL_24h_lagged',
       'AIL_2day_lagged', 'AIL_3day_lagged', 'AIL_4day_lagged',
       'AIL_5day_lagged', 'AIL_6day_lagged', 'AIL_oneweek_lagged'],
      dtype='object')

In [30]:
import numpy as np

from xgboost import XGBRegressor
from sklearn.model_selection import TimeSeriesSplit
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
import pickle
import hyperopt
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error


df = df.interpolate(method='nearest').ffill().bfill()
#df.isnull().sum()




#file_name   =  "feature_v2.pkl"
#open_file   =  open(file_name, "rb")
#loaded_list =  pickle.load(open_file)
#open_file.close()

df_hypopt       = df
split_date      = '2020-01-01 00:00:00'
df_hypopt_train = df_hypopt.loc[df_hypopt.Datetime < split_date].copy()
df_hypopt_test  = df_hypopt.loc[df_hypopt.Datetime >= split_date].copy()

y_train         = df_hypopt_train['AIL'].values
y_test          = df_hypopt_test['AIL'].values
X_train         = df_hypopt_train.drop(['Datetime', 'AIL', 'just_date'], axis = 1).values
X_test          = df_hypopt_test.drop (['Datetime', 'AIL', 'just_date'], axis = 1).values



space={ 'max_depth'        : hp.quniform("max_depth", 2, 12, 1),
        'gamma'            : hp.uniform ('gamma', 0,1),
        'reg_alpha'        : hp.quniform('reg_alpha', 0,2,1),
        'reg_lambda'       : hp.uniform('reg_lambda', 0,1),
        'colsample_bytree' : hp.uniform('colsample_bytree', 0.5,1),
        'min_child_weight' : hp.quniform('min_child_weight', 1, 7, 1),
        'n_estimators'     : hp.uniform('n_estimators', 10, 800)
    }


#def mape(pred, true):
#    return np.abs(np.asarray(pred) - np.asarray(true)) / np.maximum(np.abs(np.assarry(true))

delta = 10 #huber-delta

def optimize (space):
    model=XGBRegressor(
                       n_estimators     = int(space['n_estimators']), 
                       max_depth        = int(space['max_depth']),
                       gamma            = space['gamma'],
                       reg_alpha        = int(space['reg_alpha']),
                       min_child_weight = space['min_child_weight'],
                       colsample_bytree = space['colsample_bytree'])
    
    model.fit(X_train, y_train)
    # summarize performance
    preds     = model.predict(X_test)
    #mae       = mean_absolute_error(y_test, preds)
    #loss = np.where(np.abs(y_test - preds) < delta, 
    #                0.5*((y_test - preds)**2),
    #                delta*np.abs(y_test-preds) - 0.5*(delta**2)
    #               )
    loss = np.log(np.cosh(preds- y_test))
    return np.sum(loss)
    
trials = Trials()
result = fmin(
            fn        = optimize,
            space     = space,
            algo      = tpe.suggest,
            max_evals = 100,
            trials    = trials
)

print (result)

  1%|          | 1/100 [00:00<00:28,  3.45trial/s, best loss: inf]

/Users/abulhasanfahad/opt/anaconda3/envs/ml/lib/python3.7/site-packages/ipykernel_launcher.py:70: RuntimeWarning: overflow encountered in cosh



100%|██████████| 100/100 [09:40<00:00,  5.80s/trial, best loss: 436063.6158290594]
{'colsample_bytree': 0.8207757988334056, 'gamma': 0.76701931974563, 'max_depth': 3.0, 'min_child_weight': 4.0, 'n_estimators': 696.8672983981303, 'reg_alpha': 2.0, 'reg_lambda': 0.9899167896435772}


In [27]:
import numpy as np
import pandas as pd
from xgboost import XGBRegressor
from datetime import datetime
import joblib
from joblib import dump

column_list = ['hour_of_day', 'off_peak', 'on_peak', 
       'day', 'sin.day', 'cos.day', 'sin.hour', 'cos.hour', 'weekend',
       'monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday',
       'sunday', 'month_0', 'month_1', 'month_2', 'month_3', 'month_4',
       'month_5', 'month_6', 'month_7', 'month_8', 'month_9', 'month_10',
       'month_11', 'hour_0', 'hour_1', 'hour_2', 'hour_3', 'hour_4', 'hour_5',
       'hour_6', 'hour_7', 'hour_8', 'hour_9', 'hour_10', 'hour_11', 'hour_12',
       'hour_13', 'hour_14', 'hour_15', 'hour_16', 'hour_17', 'hour_18',
       'hour_19', 'hour_20', 'hour_21', 'hour_22', 'hour_23', 'year',
       'sunlight_avaialbility', 'AIL_previous_hour', 'AIL_24h_lagged',
       'AIL_2day_lagged', 'AIL_3day_lagged', 'AIL_4day_lagged',
       'AIL_5day_lagged', 'AIL_6day_lagged', 'AIL_oneweek_lagged' ]

df_final = pd.DataFrame(data = df)
df_final = df_final.interpolate(method='nearest').ffill().bfill()

y_train = pd.DataFrame(df, columns = ['AIL'])
y_train = y_train.values
X_train = pd.DataFrame(df, columns = column_list)
X_train = X_train.values

'''
mae loss: 'colsample_bytree': 0.9109339305529397, 'gamma': 0.6461068249891915, 'max_depth': 3.0, 'min_child_weight': 5.0, 
'n_estimators': 717.5962834481896, 'reg_alpha': 2.0, 'reg_lambda': 0.8878703412755587}
'''
'''
huber_loss:

'colsample_bytree': 0.7992205992647909, 'gamma': 0.5783448371612743, 'max_depth': 3.0, 
'min_child_weight': 2.0, 'n_estimators': 739.331766781355,
'reg_alpha': 2.0, 'reg_lambda': 0.7732711356862325

'''

model   = XGBRegressor(colsample_bytree = 0.7992205992647909, 
                       gamma            = 0.5783448371612743, 
                       max_depth        = 3, 
                       min_child_weight = 2, 
                       n_estimators     = 739, 
                       reg_alpha        = 2, 
                       reg_lambda       = 0.7732711356862325)

model.fit(X_train,y_train)
preds = model.predict(X_train)
print(mean_absolute_error(y_train, preds))

# save model to file
model_name = 'model_generated_on_{}_{}_{}_v6_non_holiday_huber_loss.joblib.dat'.format(datetime.now().day, datetime.now().month, datetime.now().year)
joblib.dump(model, filename = model_name)

30.731405857055986


['model_generated_on_13_2_2021_v6_non_holiday_huber_loss.joblib.dat']

In [7]:
import numpy as np

from   xgboost import XGBRegressor
#from sklearn.model_selection import TimeSeriesSplit
#from   matplotlib import pyplot as plt
#from   sklearn.model_selection import train_test_split
#from   sklearn.preprocessing import MinMaxScaler
import sklearn
from   datetime import datetime
import pickle
import hyperopt
from   hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from   sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from   hpsklearn import HyperoptEstimator, any_preprocessing, any_regressor
from hpsklearn import gradient_boosting_regression

from   hyperopt import tpe


df = df.interpolate(method='nearest').ffill().bfill()
#df.isnull().sum()




#file_name   =  "feature_v2.pkl"
#open_file   =  open(file_name, "rb")
#loaded_list =  pickle.load(open_file)
#open_file.close()

df_hypopt       = df
split_date      = '2020-01-01 00:00:00'
df_hypopt_train = df_hypopt.loc[df_hypopt.Datetime < split_date].copy()
df_hypopt_test  = df_hypopt.loc[df_hypopt.Datetime >= split_date].copy()

y_train         = df_hypopt_train['AIL'].values
y_test          = df_hypopt_test['AIL'].values
X_train         = df_hypopt_train.drop(['Datetime', 'AIL', 'just_date'], axis = 1).values
X_test          = df_hypopt_test.drop (['Datetime', 'AIL', 'just_date'], axis = 1).values


# define search
model = HyperoptEstimator(regressor     = gradient_boosting_regression('gbr'), 
                          preprocessing = any_preprocessing('pre'), 
                          loss_fn       = mean_absolute_error, 
                          algo          = tpe.suggest, 
                          max_evals     = 50, 
                          trial_timeout  = 30)
# perform the search
model.fit(X_train, y_train)
# summarize performance
mae = model.score(X_test, y_test)
print("MAE: %.3f" % mae)
# summarize the best model
print(model.best_model())

  0%|          | 0/1 [00:00<?, ?trial/s, best loss=?]

/Users/abulhasanfahad/opt/anaconda3/envs/ml/lib/python3.7/site-packages/sklearn/ensemble/_gb.py:309: FutureWarning: The parameter 'presort' is deprecated and has no effect. It will be removed in v0.24. You can suppress this warning by not passing any value to the 'presort' parameter. We also recommend using HistGradientBoosting models instead.
  FutureWarning)



 50%|█████     | 1/2 [00:00<?, ?trial/s, best loss=?]

/Users/abulhasanfahad/opt/anaconda3/envs/ml/lib/python3.7/site-packages/sklearn/ensemble/_gb.py:309: FutureWarning: The parameter 'presort' is deprecated and has no effect. It will be removed in v0.24. You can suppress this warning by not passing any value to the 'presort' parameter. We also recommend using HistGradientBoosting models instead.
  FutureWarning)



 67%|██████▋   | 2/3 [00:00<?, ?trial/s, best loss=?]

...

 98%|█████████▊| 49/50 [00:00<?, ?trial/s, best loss=?]

/Users/abulhasanfahad/opt/anaconda3/envs/ml/lib/python3.7/site-packages/sklearn/ensemble/_gb.py:309: FutureWarning: The parameter 'presort' is deprecated and has no effect. It will be removed in v0.24. You can suppress this warning by not passing any value to the 'presort' parameter. We also recommend using HistGradientBoosting models instead.
  FutureWarning)



100%|██████████| 50/50 [00:04<00:00, 11.27trial/s, best loss: 57.0328007954641]


/Users/abulhasanfahad/opt/anaconda3/envs/ml/lib/python3.7/site-packages/sklearn/ensemble/_gb.py:309: FutureWarning: The parameter 'presort' is deprecated and has no effect. It will be removed in v0.24. You can suppress this warning by not passing any value to the 'presort' parameter. We also recommend using HistGradientBoosting models instead.
  FutureWarning)


MAE: 0.991
{'learner': GradientBoostingRegressor(learning_rate=0.32941608891734564, max_depth=None,
                          n_estimators=45, presort='auto', random_state=3,
                          subsample=0.7719952128201714), 'preprocs': (), 'ex_preprocs': ()}
